In [ ]:
import sys
import pickle
import pprint
pp = pprint.PrettyPrinter(indent=0)

import numpy as np
from functools import reduce

# Local Modules

### utilities.py

In [ ]:
# utilties.py

def load_behavioral_profile(fname = None): pass

def load_spirometer_profile(fname = None): pass

def load_values_profile(fname = None): pass

def load_skill_cooccurance(fname = './data/skill_occurance.pickle'):
    with open(fname, 'rb') as f:
        projects = pickle.load(f)
    return projects

def load_social_media_profiles(fname = './data/twitter_profile.pickle'):
    with open(fname, 'rb') as f:
        ptwitter = pickle.load(f)
    return ptwitter

def get_attributes(pobj):
    """Given any python object returns its data attributes.

    Args:
        pobj (:obj:`Maker`): A python object.

    Returns:
        (:obj:`list` of :obj:`str`): list of attribute names.

    Raises:
        TypeError: Raised if an object type without a __dict__ attribute is given.
        
    """

    if '__dict__' not in dir(pobj):
        raise TypeError('A Python object with attributes is expected.')

    attrs = {k:v for k,v in pobj.__dict__.items() if not k.startswith('_')}
    return [k for k, v in attrs.items() if type(v) != 'function']


def load_member_profiles(fname = './data/crm_profile.pickle'):
    """The utility loads member profile data.

    Args:
        pobj (:obj:`pickle`): A serialized python dictionary.

    Returns:
        (:obj:`dict` of :obj:`dict`): The python dictionary of profiles.
            The member id is a key, a profile of the member is the value.
        
    """
    with open(fname,'rb') as f:
        pcrm = pickle.load(f)
    
    return pcrm


def get_random_item(adict):
    """The function returns a random item from a dictionary.

    Args:
        pobj (:obj:`dict`): A python dictionary.

    Returns:
         None: If the the input is an empty dictionary.
        (:obj:`dict`): A random dictionary item in the form {k:v}.
        
     Raises:
        TypeError: Raised if a non `dict` is given.
    
    """
    if not isinstance(adict, dict):
        raise TypeError('A Python dictionary object is expected.') 
    N = len(adict)
    if N == 0: return
    ind = int(np.random.rand() * (N - 1))
    k,v = list(adict.items())[ind]
    return {k:v}

def get_sample(adict, n):
    """The function returns a number of random items from a dictionary.

    Args:
        pobj (:obj:`dict`): A python dictionary.
        n (:obj:`int`): A number of items.

    Returns:
         None: If the the input is an empty dictionary.
        (:obj:`dict`): A dictionary with item in the form {k:v}.
        
     Raises:
        TypeError: Raised if a non `dict` is given.
    
    """
    if not isinstance(adict, dict):
        raise TypeError('A Python dictionary object is expected.') 
    N = len(adict)
    if N == 0: return
    if n > N: n = N
    sample = dict()
    while n > 0:
        ind = int(np.random.rand() * (N - 1))
        k,v = list(adict.items())[ind]
        sample[k] = v
        n -= 1
    return sample

def form_network_relations(connections):
    """Given a list of attribute/connection sets, it returns a co-occurance matrix of the attributes
        observed in the sets.
    
    Note: It should be noted that the diagonal, which would be occurance count is empty and this frequency
        info can be accessed by the 'f' key of the matrix/dictionary.
  
    Args:
        pobj (:obj:`list` of :obj:`set` of :obj:`str`): Co-oocurance data.

    Returns:
        (:obj:`dict`): A dictionary which contains a semantic network data where the keys are the nodes (attributes)
        and values are a dictionary which contains the degree and the weighted links of the node.
    """
    attrs = connections
    alltags = set()
    for k in attrs:
        alltags = alltags.union(set(k))
        
    occurance = dict()
    for t in alltags:
        occurance[t] = {'f':0,'co':{}}
        
    for kset in attrs:
        for k in kset:
            occurance[k]['f'] += 1
            for t in kset:
                if t == k: continue
                if t in occurance[k]['co'].keys():
                    occurance[k]['co'][t] += 1
                else:
                    occurance[k]['co'][t] = 1            
    return occurance



In [ ]:
class SocialGraph(object):
    """A graph object which represents intracommunity social connections.

    Args:
        

    Attributes:
        

    """

    def __init__(self, profiles = None, media = 'Twitter'):
        self.nodes = set()
        self.connections = {}
        self.attributes = {}
        self.notmembers = set()
        if profiles: self.construct_net(profiles, media)
    
    def add_isolate(self, noid, ismember = True):
        """Adds an isolate to the graph. 

        Args:
            noid (:obj:`int`): A unique node id.
            ismember (:obj:`bool`): whether the node is a member (default True).
            
        Returns:
            (:obj:`bool`): False if node already exists, otherwise it returns True.
            

        """
        if noid in self.nodes: return False
        if noid in self.notmembers: return False
        if not ismember:
            self.notmembers.add(noid)
            return True
        self.nodes.add(noid)
        self.connections.update({noid: {'bi': set(),
                                  'in': set(),
                                  'out': set(),
                                  'ext': set()
                                 }
                                })
        return True
    
    
    def remove_node(self, noid):
        """Not implemented yet.
        
        """
        pass
    
    def update_attributes(self, noid, attributes):
        """Updates the attributes of the node.

        Args:
            noid (:obj:`int`): A unique node id 
            attributes (:obj:`dict`): A dictionary with new attributes.
            
        Returns:
            (:obj:`bool`): Returns True.
            

        """
        if not noid in self.attributes.keys():
            self.attributes[noid] = attributes
            # It is very likely that the node is not in the network
            self.add_isolate(noid)
            return True
        for attr, val in attributes.items():
            self.attributes[noid][attr] = val
        return True
    
    def get_attribute(self, noid, attr):
        """Retrieves the value of the specific attribute of the node.

        Args:
            noid (:obj:`int`): A unique node id 
            attr (:obj:`str`): An attribute.
            
        Returns:
            (:obj:`obj`): If attribute exists.
            None: If attributes does not exist
            

        """
        if not noid in self.attributes.keys(): return
        if not attr in self.attributes[noid].keys(): return   
        return self.attributes[noid][attr]
    
    def get_noid(self, query):
        """Retrieves the nodeid for which the attribute matches.

        Args:
            noid (:obj:`int`): A unique node id 
            query (:obj:`tuple`): An attribute and value tuple.
            
        Returns:
            (:obj:`int`): The node id of the first attribute match.
            None: If query doesn't match to any node.
            
        """
        attr,val = query
        print('get_noid:')
        for noid in self.attributes.keys():
            if not attr in self.attributes[noid].keys(): continue
            print('noid: {}, query: {}, actual: {}'.format(noid, val, self.attributes[noid][attr]))
            if self.attributes[noid][attr] == val: return noid
        return
            
    
    def add_link(self, start, end, isbi = False):
        """Adds a link to the social graph.

        Args:
            start (:obj:`int`): A unique node id 
            end (:obj:`int`): A unique node id
            isbi (:obj:`bool`): whether the link is a bidirectional (default False).
            
        Returns:
            (:obj:`bool`): Returns True.
            

        """
        if start in self.nodes and end in self.nodes:
            if isbi:
                self.connections[start]['bi'].add(end)
                self.connections[start]['in'].discard(end)
                self.connections[start]['out'].discard(end)
                self.connections[end]['bi'].add(start)
                self.connections[end]['in'].discard(start)
                self.connections[end]['out'].discard(start)
            elif end in self.connections[start]['in']:
                self.connections[start]['bi'].add(end)
                self.connections[end]['bi'].add(start)
                self.connections[start]['in'].discard(end)
                self.connections[end]['out'].discard(start)
            else:
                self.connections[start]['out'].add(end)
                self.connections[end]['in'].add(start)
        elif start in self.nodes:
            self.connections[start]['ext'].add(end)
            self.notmembers.add(end)
        elif end in self.nodes:
            self.connections[end]['ext'].add(start)
            self.notmembers.add(start)
        else:
            self.notmembers.add(start)
            self.notmembers.add(end)
        return True
    
    
    def remove_link(self, start, end, isbi = False):
        """Not implemented yet.
        
        """
        pass
    
    def extend_net(self, profiles, media = 'Twitter'):
        """Given an external connection data, the method populates
        the SocialGraph instance in a batch mode.
        
        Note:
            Only a social graph from Twitter data is supported at the moment.

        Args:
            profiles (:obj:`dict`): A json like dict item where used keys are member ids
            and each value is a :obj:`dict` whose keys in return are
                - 'TwitterId': twitter ID
                - 'nf' : number of friends
                - 'nfollowers': number of followers
                - 'flist': list of friends' twitter IDs.
            
        Returns:
            (:obj:`bool`): False if node already exists, otherwise it returns True.
            

        """
        mids = profiles.keys()
        for m in mids:
            self.add_isolate(m)
            self.update_attributes(m,
                                   {'TwitterId': int(profiles[m]['TwitterId']),
                                    #'nfollowers': profiles[m]['nfollowers'],
                                    'nfriends': int(profiles[m]['nf'])
                                   }
                                  )
        for from_id in mids:
            for to_id in mids:
                if from_id == to_id: continue
                ifrom_id = int(profiles[from_id]['TwitterId'])
                ito_id = int(profiles[to_id]['TwitterId'])
                afriends = profiles[from_id]['flist']
                bfriends = profiles[to_id]['flist']
                ab = ito_id in afriends
                ba = ifrom_id in bfriends
                if ab and ba: self.add_link(from_id, to_id, isbi=True)
                elif ab: self.add_link(from_id, to_id)
                elif ba: self.add_link(to_id, from_id)
                else: pass
        media_ids = set([v['TwitterId'] for k, v in self.attributes.items()])
        for m in mids:
            friends = set(profiles[m]['flist'])
            ext = friends.difference(media_ids)
            self.connections[m]['ext'] = ext
            self.notmembers = self.notmembers.union(ext)
        return True 
    
    def get_community_net(self):
        """Not implemented yet.
        
        """
        pass

In [ ]:
SG = SocialGraph()
print(get_attributes(SG))

In [ ]:
print(SG.nodes)
print(SG.notmembers)
print(SG.connections)
print(SG.attributes)

In [ ]:
TWITTER_PROFILES = load_social_media_profiles('./data/twitter_profile.pickle')

In [ ]:
SAMPLE = get_sample(TWITTER_PROFILES, 20)

In [ ]:
SG.extend_net(SAMPLE)

In [ ]:
pp.pprint(SG.attributes)

In [ ]:
print(len(SG.notmembers))
print(len(SG.nodes))

In [ ]:
SG.add_link('158526065','154653833')

In [ ]:
for m,v in SG.connections.items():
    print("ID = ", m)
    print('Bi = ', v['bi'])
    print('In = ', v['in'])
    print('Out = ', v['out'])
    print('nExt =', len(v['ext']))

In [ ]:
coMat = load_skill_cooccurance()
net = form_network_relations(coMat)
pp.pprint(net)

### agents.py

In [ ]:
# agents.py
from six import string_types

class Agent(object):
    """A generic agent class.

    Args:
        id (:obj:`str`)): A unique identifier for the agent.
        state (:obj:`str`, optional): Current state of the agent.

    Attributes:
        id (:obj:`int`)): A unique identifier for the agent.
        state (:obj:`str`, optional): Current state of the agent.

    """

    def __init__(self, aid, state = 'idle', atype = 'generic'):
        self.id = aid
        self.state = state
        self.type = atype

    def current_state(self):
        """It reports the current state of the agent.

        Returns:
            (:obj:`dict`): A `dict` summarizing the state of the agent.

        """

        state = {'id': self.id, 'state': self.state, 'type': self.type}
        return state
    


class Maker(Agent):
    """A maker (:obj:`Agent`) class.

        Notes:


        Attributes:
            name (:obj:`str`, optional): Name of the maker.
            surname (:obj:`str`, optional): Surname of the maker.
            twitter (:obj:`str`, optional): Twitter name of the maker.
            omid (:obj:`int`, optional): OpenMaker platform id of the maker.
            location (:obj:`str`, optional): The geogrpahic location of the maker.
            entity (:obj:`str`, optional): Whether the agent is one of organization 'O' or person 'P'.
            gender (:obj:`str`, optional): The gender is one of 'M', 'F', and 'X.
            tags (:obj:`set` of :obj:`str`, optional): Self assigned tags of the maker.
            skills (:obj:`set` of :obj:`str`, optional): Skills of the agent.
            technologies(:obj:`set` of :obj:`str`, optional): Technologies used by the agent.
            areas (:obj:`set` of :obj:`str`, optional): Interest areas in terms technologies.
            domains (:obj:`set` of :obj:`str`, optional): Activity domains of the agent.

    """
    name = ''
    surname = ''
    twitter = ''
    entity = ''
    location = ''
    gender = ''
    tags = set()
    skills = set()
    technologies = set()
    areas = set()
    domains = set()

    def __init__(self, aid, expertise_model, state='idle'):
        super().__init__(aid, state, atype = 'Maker')
        self.modCogExp =  expertise_model

    def load_profile(self, profile):
        """Updates or loads the profile od the agent.

            Args:
                profile (:obj:`dict`): A python dictionary where keys are attributes of
                this class data attributes and values are the values to be loaded.

            Returns:
                (obj:`bool`): when successful.

        """
        attrs = get_attributes(self)
        attrs.extend(get_attributes(Maker))
        for k,v in profile.items():
            if k not in attrs: continue
            if isinstance(v, string_types): v='"'+str(v)+'"'
            exec("self.{} = {}".format(k,v))

        return True
    
    def skill_similarity(self, skills):
        """Agent's own declared skills to the given an external skill set, agent
            measures the similarity of the given profile to her profile using the knowledge 
            the agent has about the profiles in the community as a reference.
        
        Note: It should be noted that the shared common skills as well as resemblance of differing
            skillsets is added to the score.
            
        Args:
            skills (:obj:`list` of :obj:`str`): The skill set of the other agent.
            
        Returns:
            (:obj:`float`): A similarity score.
            
        """
        f = self.modCogExp.get_resemblance
        # Agent uses its own version of expertise model.
        theother = set(skills)
        common = self.skills.intersection(theother)
        dif_other = theother.difference(common)
        dif_self = self.skills.difference(common)
        lo = list(dif_other) * len(dif_self)
        ls = np.repeat(list(dif_self), len(dif_other))
        nc = len(common)
        return reduce(lambda x,y: x + y if x else y, map(lambda x,y: f(x,y,source='profiles'),lo,ls), nc)
        
    
    def skill_compatibility(self, skills):
        """Comparing agent's own declared skills to the given an external profile, agent
            measure the compatibility of the external profile to her profile considering
            the knowledge the agent has about the projects in the community.
        
        Args:
            skills (:obj:`list` of :obj:`str`): The skill set of the other agent.
            
        Returns:
            (:obj:`float`): The compatibility score.
            
        """
        f = self.modCogExp.get_compatibility
        theother = set(skills)
        common = self.skills.intersection(theother)
        dif_other = theother.difference(common)
        dif_self = self.skills.difference(common)
        lo = list(dif_other) * len(dif_self)
        ls = np.repeat(list(dif_self), len(dif_other))
        return reduce(lambda x,y: x + y if x else y, map(lambda x,y: f(x,y), lo, ls), len(common)) 
            
    
    def skill_difference(self, skills):
        """Agent's compares her own declared skills to the given an external profile measuring
            the difference of the external profile to her profile considering the knowledge the agent has
            about the projects in the community.
        
        Note: In this implementation for a skill that the other has and the self doesn't, the distincness of
            that skill in the community is used as to weight the differing value of that skill.
        
        Args:
            skills (:obj:`list` of :obj:`str`): The skill set of the other agent.
            
        Returns:
            (:obj:`float`): The skill difference score.
            
        """
        theother = set(skills)
        common = self.skills.intersection(theother)
        dif_other = theother.difference(common) 
        score = 0
        for o in dif_other:
            score_o = 0
            for s in self.skills:
                score_o += self.modCogExp.get_difference(o,s,raw=False,exclusive=False)
            w_o = self.modCogExp.get_distinctiveness(o)
            score += w_o * score_o
        return score
    
    def social_resemblance(self, egonet):
        """Agent compares her social connections to the social connections of another agent. She measures her 
        structural equivalance to the other agent, both in terms of similarities as well as differences.
        
        Args:
            
        Returns:
            
        """
        pass
    
    def social_difference(self, egonet):
        """Agent compares her social connections to the social connections of another agent. She measures
        social difference the other agent has. 
        
        Args:
            
        Returns:
            
        """
        pass
    

In [ ]:
coMat = load_skill_cooccurance()
CE = CognitiveExpertise(coMat,coMat)
pp.pprint(coMat)

In [ ]:
pp.pprint(CE.mat_proj['research'])

In [ ]:
m = Maker(1,CE)
MEMBER_PROFILES = load_member_profiles(fname = './data/crm_profile.pickle')
aprofile = get_a_dict_item(MEMBER_PROFILES)
aid = list(aprofile.keys())[0]
profile = aprofile[aid]
profile['id'] = aid
pp.pprint(profile)

In [ ]:
profile['skills'] = profile['tags']
m.load_profile(profile)
m.skills

In [ ]:
CE.get_compatibility('3d', '3dmodel'), CE.get_compatibility('3dmodel', '3d')

In [ ]:
CE.get_compatibility('3d', 'audio')

In [ ]:
m.skill_compatibility(['3d', 'opendesign', 'solutions'])

In [ ]:
m.skill_similarity(['3d', 'audio', 'opendesign', 'solutions'])

In [ ]:
m.skill_difference(['3d',  'opendesign', 'research', 'solutions'])

### models.py

#### Expertise    

In [ ]:
class CognitiveExpertise(object):
    """A python object that contains methods and data models where an agent be an indivudual or 
        a community constructs and adopts its perception of expertise around his environment.

    Args:
        id (:obj:`str`)): A unique identifier for the agent.
        state (:obj:`str`, optional): Current state of the agent.

    Attributes:
        id (:obj:`int`)): A unique identifier for the agent.
        state (:obj:`str`, optional): Current state of the agent.

    """

    def __init__(self, projects, profiles):
        self.nproj = len(projects)
        self.nprof = len(profiles)
      
        netproj = form_network_relations(projects)
        netprof = form_network_relations(profiles)
        
        self.tags_proj = list(netproj.keys())
        self.tags_prof = list(netprof.keys())
        
        self.tf_proj = {t:netproj[t]['f'] for t in netproj.keys()}
        self.tf_prof = {t:netprof[t]['f'] for t in netprof.keys()}
                
        self.mat_proj = {t:netproj[t]['co'] for t in netproj.keys()}
        self.mat_prof = {t:netprof[t]['co'] for t in netprof.keys()}
    
    
    def _get_cooccurances(self, a, b, tags, mat):
        """A given two skills it returns co-occurance of them.

        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.
            tags: (:obj:`list` of :obj:`str`): Holds the list of skills(column names),
            mat (:obj:`dict`): A dictionary whose values are dictionary holds
                the frequency counts.

        Returns:
            None: When either a or b is not occured.
            (:obj:`int`): The raw co-occurance frequency.

        """
        if a not in tags: return
        if b not in tags: return
        if b not in mat[a].keys(): return 0
        
        return mat[a][b]
    
    
    def get_profile_cooccurances(self, a, b):
        """A given two skills it returns co-occurance of them in profiles.

        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.

        Returns:
            None: When either a or b is not occured.
            (:obj:`int`): The raw co-occurance frequency.

        """
        return self._get_cooccurances(a, b,
                                      self.tags_prof,
                                     self.mat_prof) 
    
    
    def get_project_cooccurances(self, a, b):
        """A given two skills it returns co-occurance of them in projects.

        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.

        Returns:
            None: When either a or b is not occured.
            (:obj:`int`): The raw co-occurance frequency.

        """
        return self._get_cooccurances(a, b,
                                      self.tags_proj,
                                      self.mat_proj) 

    
    def compute_centralities(self, raw=True):
        """Computes the centralities of skillset that occurance matrix.
        
        Note: 
            This is a mapping of skills on a co-occurance map: the more often a skill is required
            for the projects in a community the more central is the skill to the community.
        Args:
            raw (:obj:`bool`): Is used to determine whether raw count or normalized values should be returned
            (default True).

        Returns:
            (:obj:`dict`): A dictionary holding the centrality of each skill.
            
        """
        self.centralities = dict()
        for t in self.tags_proj:
            self.centralities[t] = self.get_centrality(t,raw)
        
        return self.centralities
    

    def get_centrality(self, a, raw=True):
        """Returns the frequency count of a skill in co-occurance matrix formed from project requirements.
        
        Note: 
            As of the co-occurance map: the more often a skill is required for the projects in a community
            the more central is the skill to the community.
        Args:
            a (:obj:`str`): A skill identifier
            raw (:obj:`bool`): Is used whether raw count needs to be returned (default True).

        Returns:
            None: When no project info has found.
            (:obj:`int`): The raw co-occurance frequency when arg raw = True.
            (:obj:`float`): The normalized frequency count as of total project observations.
            
        """
        if a not in self.tags_proj: return 0
        if not self.nproj: return
        
        fa = self.tf_proj[a]
        if raw: return fa
        return fa / (1.0 * self.nproj)

    
    def compute_diffusions(self, raw=True):
        """Computes the diffusion of skills among the community members.
        
        Args:
            raw (:obj:`bool`): Is used to determine whether raw count or normalized values should be returned
            (default True).

        Returns:
            (:obj:`dict`): A dictionary holding the diffusion score for each skill.
            
        """
        self.diffusions = dict()
        for t in self.tags_prof:
            self.diffusions[t] = self.get_diffusion(t,raw)
        
        return self.diffusions

    
    def get_diffusion(self, a, raw=True):
        """Returns the frequency count of a skill in co-occurance matrix formed from agent profiles.

        Note: 
            As of the co-occurance map: the more often a skill is observed within a community the more
            diffused is the skill in the community.
            
        Args:
            a (:obj:`str`): A skill identifier
            raw (:obj:`bool`): Is used to determine whether raw count needs to be returned (default True).

        Returns:
            None: When no profile info has found.
            (:obj:`int`): The raw co-occurance frequency when the arg raw = True.
            (:obj:`float`): The normalized frequency count as of total profiles/members in the community.
            
        """
        if a not in self.tags_prof: return 0
        if not self.nprof: return
        
        fa = self.tf_prof[a]
        if raw: return fa
        return fa / (1.0 * self.nprof)

    
    def compute_distinctivenesses(self):
        """Computes the distinctiveness of skills within the community parctices.
        
        Returns:
            (:obj:`dict`): A dictionary holding the distinctiveness score for each skill.
            
        """
        self.distinctivenesses = dict()
        for t in self.tags_proj:
            self.distinctivenesses[t] = self.get_distinctiveness(t)
        
        return self.distinctivenesses
    
    
    def get_distinctiveness(self, a):
        """Returns a score to measure to what extend an important skill for the community is owned by less
            members.
            
        Args:
            a (:obj:`str`): A skill identifier
            
        Returns:
            None: When no profile info has found.
            (:obj:`int`): The raw co-occurance frequency when the arg raw = True.
            (:obj:`float`): The normalized frequency count as of total profiles/members in the community.
            
        """
        x = self.get_diffusion(a)
        y = self.get_centrality(a)
        if not x: return
        if y is None: return 
        return y / x


    def get_union(self, a, b, source='projects'):
        """Returns total occurances where a or b is observed.
        
        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.
            source (:obj:`str`): Denotes the type of data source.
                It can be either 'projects' or 'profiles' (default 'projects')

        Returns:
            None: When either a or b is not occured.
            (:obj:`int`): The magnitude of the frquency counts where either a or b has occured.

        """
        if source == 'profiles':
            fa = self.tf_prof[a]
            fb = self.tf_prof[b]
            fab = self.get_project_cooccurances(a,b)
        else:
            fa = self.tf_proj[a]
            fb = self.tf_proj[b]
            fab = self.get_profile_cooccurances(a,b)
        return fa + fb - fab

    
    def get_difference(self, a, b, raw=True, exclusive=True, source='projects'):
        """Returns the count of occurances of a without b.
        
        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.
            source (:obj:`str`): Denotes the type of data source. It can be either 
                'projects' or 'profiles' (default 'projects')

        Returns:
            None: When either a or b is not occured.
            (:obj:`int`): The occurance frquency counts of a without b 
                when a raw count is requested.
            (:obj:`float`): The ratio of occurance of a without b with respect to frequencies of a
                or with resepct to cases where either frequencies of a or b has been observed.

        """
        if source == 'projects':
            fa = self.tf_proj[a]
            fab = self.get_project_cooccurances(a,b)
        else:
            fa = self.tf_prof[a]
            fab = self.get_profile_cooccurances(a,b)
            
        dif = fa - fab
        if raw: return dif
        if exclusive: return dif / (1.0 * fa)
        return dif / (1.0 * self.get_union(a,b,source))

    
    def get_similarity(self, a,b, raw=True, exclusive=True, source='projects'):
        """Returns the count of co-occurances of a and b.

        Note: When the union is used as the the reference, that is when exclusive is false,
            sim(a,b) = sim(b,a)
            
        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.
            source (:obj:`str`): Denotes the type of data source. It can be either 
                'projects' or 'profiles' (default 'projects')

        Returns:
            None: When either a or b is not occured.
            (:obj:`int`): The joint occurances of a and b.

        """
        if source == 'projects':
            fab = self.get_project_cooccurances(a,b)
            fa = self.tf_proj[a]
        else:
            fab = self.get_profile_cooccurances(a,b)
            fa = self.tf_prof[a]
            
        if raw: return fab
        if exclusive: return fab / (1.0 * fa)
        return fab / (1.0 * self.get_union(a,b,source))

    
    def get_resemblance(self, a, b, source='projects'):
        """Returns the resemblance of a and b in terms of both their similarities and differences.
        
        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.
            source (:obj:`str`): Denotes the type of data source. It can be either 
                'projects' or 'profiles' (default 'projects')

        Returns:
            None: When either a or b is not occured.
            (:obj:`float`): The resemblance ratio.

        """
        if source == 'projects':
            fab = self.get_project_cooccurances(a,b)
        else:
            fab = self.get_profile_cooccurances(a,b)
            
        dif_ab = self.get_difference(a,b,raw=False,source=source)
        dif_ba = self.get_difference(b,a,raw=False,source=source)
        sim_ab = self.get_similarity(a,b,raw=False,exclusive=False,source=source)
        return sim_ab * (1 - abs(dif_ab - dif_ba))

    
    def get_compatibility(self, a, b, source='projects'):
        """Returns a score to measure the compatibility of `a` to `b`, which is,
            given all occurances of skill `b` it measures to what extend skill `a` cooccurs
            along with skill `b`.

        Note: If, for instance, skill b happens to be occuring only when skill `a` occurs
            then the compatibility of `a` to `b` would be 1. In an asymmetric compatibility
            (that is `a` to `b` VS  `b` to `a`) where one of the compatibilities is almost 1 or 1,
            this may point out a  skill and sub-skill relation.
        
        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.
            source (:obj:`str`): Denotes the type of data source. It can be either 
                'projects' or 'profiles' (default 'projects')

        Returns:
            None: When either a or b is not occured.
            (:obj:`float`): The compatibility score of skill `a` to skill `b`.

        """
        return self.get_similarity(b, a, raw=False, exclusive=True,source=source)
    
  
    def update_relevance_models(self, new_info):
        """Given new external project information, the module updates the data model
            that is used to compute expertise relations for an agent or for a community.
        
        Args:
            new_info is one of
                (:obj:`list` of :obj:`str`): A list of skills that appeared on a project requirement.
                (:obj:`list` of :obj:`list` of :obj:`str`): A list of skill co-occurances.

        Returns:
            (:obj:`bool`): Notifying the completion of learning.
        
        """
        if not new_info: return
        if type(new_info[0]) is list: 
            new_projects = [set(s) for s in new_info]
        else:
            new_projects = [set(new_info)]
        for aset in new_projects:
            self.nproj += 1 
            for a in aset:
                if a not in self.tags_proj:
                    self.tags_proj.append(a)
                    self.tf_proj[a] = 0
                    self.mat_proj[a] = {}
                self.tf_proj[a] += 1
                for b in aset:
                    if a == b: continue
                    if b in self.mat_proj[a].keys():
                        self.mat_proj[a][b] += 1
                    else:
                        self.mat_proj[a][b] = 1
        self.compute_centralities()
        self.compute_distinctivenesses()
        return True
        
    
    def update_diffusion_levels(self, new_profiles):
        """Given new profile information, the module updates the data model
            that is used to compute diffusion of skills in the community or
            around the ego network of an agent.
            
        Note: The compuation of this behaviour and the behaviour regarding skill relations 
            update is very similar, where a computational abstraction via paramterization may be
            considered at a final iteration. 
        
        Args:
            new_prfiles is one of
                (:obj:`list` of :obj:`str`): A list of skills that appeared on a prosile.
                (:obj:`list` of :obj:`list` of :obj:`str`): A list of skill co-occurances on profiles.

        Returns:
            (:obj:`bool`): Notifying the completion of update on the skill carriers.
        
        """
        
        if not new_profiles: return
        if type(new_profiles[0]) is list: 
            new_profiles = [set(s) for s in new_profiles]
        else:
            new_profiles = [set(new_profiles)]
        for aset in new_profiles:
            self.nprof += 1 
            for a in aset:
                if a not in self.tags_prof:
                    self.tags_prof.append(a)
                    self.tf_prof[a] = 0
                    self.mat_prof[a] = {}
                self.tf_prof[a] += 1
                for b in aset:
                    if a == b: continue
                    if b in self.mat_prof[a].keys():
                        self.mat_prof[a][b] += 1
                    else:
                        self.mat_prof[a][b] = 1
        self.compute_diffusions()
        self.compute_distinctivenesses()
        return True
    

In [ ]:
coMat = load_skill_cooccurance()
CE = CognitiveExpertise(coMat,coMat)
pp.pprint(CE.mat_proj)

In [ ]:
CE.nproj

In [ ]:
CE.tf_proj

In [ ]:
CE.tags_proj

In [ ]:
a = '3d'
b = '3dmodel'

In [ ]:
CE.get_centrality(a), CE.get_centrality(b)

In [ ]:
CE.get_diffusion(a), CE.get_diffusion(b)

In [ ]:
CE.get_distinctiveness(a), CE.get_distinctiveness(b)

In [ ]:
CE.get_compatibility(a, b), CE.get_compatibility(b, a)

In [ ]:
CE.get_similarity(a,b,raw=False,exclusive=False), CE.get_similarity(b, a, raw=False,exclusive=False)

In [ ]:
CE.get_similarity(a,b,raw=False,exclusive=True), CE.get_similarity(b,a,raw=False,exclusive=True)

In [ ]:
CE.get_difference(a,b,raw=False,exclusive=True), CE.get_difference(b,a,raw=False,exclusive=True)

In [ ]:
CE.get_difference(a,b,raw=False,exclusive=False), CE.get_difference(b,a,raw=False,exclusive=False)

In [ ]:
CE.get_resemblance(a, b)

In [ ]:
c = '4d'
newprojs = [[b, a], [c, a]]
CE.update_relevance_models(newprojs)

In [ ]:
print(CE.get_centrality(a), CE.get_centrality(b))
CE.get_similarity(a,b,raw=False,exclusive=True), CE.get_similarity(b,a,raw=False,exclusive=True)

In [ ]:
CE.get_compatibility(c, a), CE.get_compatibility(c, b)

In [ ]:
CE.get_centrality(a), CE.get_centrality(b), CE.get_centrality(c)

In [ ]:
CE.get_distinctiveness(a), CE.get_distinctiveness(b), CE.get_distinctiveness(c)

In [ ]:
newprofile = [a,b,c]
CE.update_diffusion_levels(newprofile)

In [ ]:
CE.get_distinctiveness(a), CE.get_distinctiveness(b), CE.get_distinctiveness(c)

### interfaces.py

In [ ]:
abs(2 - 3)

# Tests

In [ ]:
m = Maker(1)
m.load_profile({'name': 'bulent'})

In [ ]:
m.load_profile({'name': 'bulent'})

In [ ]:
m.name

In [ ]:
MEMBER_PROFILES = load_member_profiles(fname = './data/crm_profile.pickle')

In [ ]:
m = Maker(1)
m.load_profile({'name': 'bulent'})
m.name
MEMBER_PROFILES = load_member_profiles(fname = './data/crm_profile.pickle')
aprofile = get_a_dict_item(MEMBER_PROFILES)
aid = list(aprofile.keys())[0]
profile = aprofile[aid]
profile['id'] = aid
pp.pprint(profile)

In [ ]:
m.load_profile(profile)

In [ ]:
type(MEMBER_PROFILES)

In [ ]:
print(m.gender)